**환율자료 확보**
+ 수출입은행 : [Link](https://www.koreaexim.go.kr/ir/HPHKIR020M01?apino=2&viewtype=C&searchselect=&searchword=)
  + 접근 : token 발급이 용의함
  + 이슈 : 11시 자료 미업로드 됨


+ 하나은행 : [Link](https://www.hanafnapimarket.com/)
  + 접근 : token 발급이 불가?


In [18]:
# 자료 수집 현황
import pandas as pd
# from datetime import datetime, timedelta
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./GOOGLE_APPLICATION_CREDENTIALS.json"
client = bigquery.Client()

sql = """SELECT FORMAT_DATETIME('%y/%m', CAST(DATE AS DATETIME)) as YYYYMM
         , min(FORMAT_DATETIME('%y/%m/%d', CAST(DATE AS DATETIME))) as min_date
         , max(FORMAT_DATETIME('%y/%m/%d', CAST(DATE AS DATETIME))) as max_date
         FROM `boxwood-**************.openAPI.exchange_price`
         group by FORMAT_DATETIME('%y/%m', CAST(DATE AS DATETIME))"""
df1 = client.query(sql).to_dataframe()

In [19]:
df1

,YYYYMM,min_date,max_date
0,22/11,22/11/01,22/11/22
1,22/10,22/10/04,22/10/31
2,22/09,22/09/30,22/09/30


In [1]:
AUTHKEY = '__AUTHKEY__'
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey='+AUTHKEY+'&data=AP01'

In [31]:
###############
# conn test
#####

import requests
import re
from pandas import json_normalize
import pandas as pd
from datetime import datetime, timedelta
import time

AUTHKEY = '__AUTHKEY__'

date_value = '20220913'

try:
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey='+AUTHKEY+'&searchdate='+date_value+'&data=AP01'
    lm_json = requests.get(url).json()
except:
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey='+AUTHKEY+'&data=AP01'
    lm_json = requests.get(url).json()

DATE = (datetime.today() + timedelta(hours=0) - timedelta(0))
row= { 'date': DATE }
for data_ in lm_json:
    data_['ttb'] = data_['ttb'].replace(',', '')
    data_['tts'] = data_['tts'].replace(',', '')
    data_['deal_bas_r'] = data_['deal_bas_r'].replace(',', '')
    data_['bkpr'] = data_['bkpr'].replace(',', '')
    data_['kftc_bkpr'] = data_['kftc_bkpr'].replace(',', '')
    data_['kftc_deal_bas_r'] = data_['kftc_deal_bas_r'].replace(',', '')
    data_.update(row)

In [32]:
pd.DataFrame(lm_json)

,result,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm,date
0,1,AED,372.26,379.79,376.03,376,0,0,376,376.03,아랍에미리트 디르함,2022-11-23 11:18:42.151876
1,1,AUD,942.6,961.65,952.13,952,0,0,952,952.13,호주 달러,2022-11-23 11:18:42.151876
2,1,BHD,3626.83,3700.1,3663.47,3663,0,0,3663,3663.47,바레인 디나르,2022-11-23 11:18:42.151876
3,1,BND,979.29,999.08,989.19,989,0,0,989,989.19,브루나이 달러,2022-11-23 11:18:42.151876
4,1,CAD,1053.17,1074.44,1063.81,1063,0,0,1063,1063.81,캐나다 달러,2022-11-23 11:18:42.151876
5,1,CHF,1434.22,1463.19,1448.71,1448,0,0,1448,1448.71,스위스 프랑,2022-11-23 11:18:42.151876
6,1,CNH,196.28,200.25,198.27,198,0,0,198,198.27,위안화,2022-11-23 11:18:42.151876
7,1,DKK,186.17,189.94,188.06,188,0,0,188,188.06,덴마아크 크로네,2022-11-23 11:18:42.151876
8,1,EUR,1384.48,1412.45,1398.47,1398,0,0,1398,1398.47,유로,2022-11-23 11:18:42.151876
9,1,GBP,1597.79,1630.06,1613.93,1613,0,0,1613,1613.93,영국 파운드,2022-11-23 11:18:42.151876


In [16]:
###############
# main.py
#####

import pandas as pd
from datetime import datetime, timedelta
import requests
from google.cloud import bigquery

try:
    import os
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./GOOGLE_APPLICATION_CREDENTIALS.json"
    client = bigquery.Client()
except:
    pass

client = bigquery.Client('boxwood-**************')
dataset_id = 'openAPI'
table_id = 'exchange_price'
table_ref = client.dataset(dataset_id).table(table_id)
Table = client.get_table(table_ref)  # API request

def exchange_price(date_value):
    AUTHKEY = '__AUTHKEY__'
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey='+AUTHKEY+'&searchdate='+date_value+'&data=AP01'
    lm_json = requests.get(url).json()
    if date_value != '':
        DATE = datetime.strptime(date_value,'%Y%m%d').strftime('%Y-%m-%d %H:%M:%S')
    else:
        DATE = (datetime.today() + timedelta(hours=0) - timedelta(0)).strftime('%Y-%m-%d %H:%M:%S')
    for data_ in lm_json:
        cur_unit = data_['cur_unit']
        cur_nm = data_['cur_nm']
        ttb = data_['ttb'].replace(',', '')
        tts = data_['tts'].replace(',', '')
        deal_bas_r = data_['deal_bas_r'].replace(',', '')
        bkpr = data_['bkpr'].replace(',', '')
        kftc_bkpr = data_['kftc_bkpr'].replace(',', '')
        kftc_deal_bas_r = data_['kftc_deal_bas_r'].replace(',', '')
        rows_to_insert = [( cur_unit, cur_nm, float(ttb), float(tts), float(deal_bas_r), float(bkpr), float(kftc_bkpr), float(kftc_deal_bas_r),  DATE)]
        client.insert_rows(Table, rows_to_insert)
    return f'Jobs Done!'

In [44]:
exchange_price('20220829')

'jobs Done'